In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/colab_notebooks/nodeNcode/socialGraph/

/content/drive/MyDrive/colab_notebooks/nodeNcode/socialGraph


In [3]:
!pip install fastcore -Uqq

     |████████████████████████████████| 56 kB 2.4 MB/s 


In [4]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import threading
import pandas as pd
from fastcore.all import *

# Scraping

In [ ]:
def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup


def get_main_topics(soup):
    tags = {}
    for c in soup.find_all('td'):
      tags[c.text] = c.a['href']

    return tags


def get_messages_url(topic_soup):
    message = []
    for e in topic_soup.find_all('li',attrs={'class':'disc-smr cardb'}):
        message.append(e.a['href'])
    return message


def get_message(mess):
    try:
        mess_soup = get_soup(main_url+mess)
        txt = mess_soup.find_all('div', attrs={'id':'post_content'})[0].text
        return txt, topic[0]

    except:
        #logs.append((topic[0], topic_pages_url))
        pass

In [ ]:
pagespertopic = 4
main_url = 'https://patient.info'


for char in ['p', 'u', 'v', 'w', 'x', 'y', 'z']:
    pairs = []
    logs = []
    print(f'Scrapping Character : {char}')
    url = f'https://patient.info/forums/index-{char}'

    soup = get_soup(url)
    main_topics = get_main_topics(soup)
    print(f'Number of topics in {char}: {len(main_topics)}')
    pbar = tqdm(main_topics.items(), position=0, leave=True)
    for topic in pbar:

        for i in range(1,pagespertopic+1):    
            topic_pages_url = f'?page={i}#group-discussions'
            topic_soup = get_soup(main_url+topic[1]+topic_pages_url)
            message = get_messages_url(topic_soup=topic_soup)
            
            r = parallel(get_message, message, n_workers=4)
            [pairs.append(e) for e in r]
        pbar.set_description(topic[0])

    pd.DataFrame(pairs, columns=['text', 'topic']).to_csv(f'socgraph_{char}.csv')
    #pd.DataFrame(pairs, columns=['text', 'topic']).to_csv(f'socgraph_{char}.csv')

Scrapping Character : p
Number of topics in p: 127


Pyrexia of Unknown Origin (PUO): 100%|██████████| 127/127 [57:11<00:00, 27.02s/it]


Scrapping Character : u
Number of topics in u: 17


Utovlan: 100%|██████████| 17/17 [07:28<00:00, 26.36s/it]


Scrapping Character : v
Number of topics in v: 30


...

XXY and other SCAs: 100%|██████████| 5/5 [02:18<00:00, 27.60s/it]


Scrapping Character : y
Number of topics in y: 2


Yeast Infections - Non Genital: 100%|██████████| 2/2 [00:53<00:00, 26.63s/it]


Scrapping Character : z
Number of topics in z: 14


Zyprexa: 100%|██████████| 14/14 [06:26<00:00, 27.62s/it]


# Making single dataframe

In [5]:
import string
alp = string.ascii_lowercase

In [12]:
df = pd.DataFrame()
logs = []
for char in alp:
    try:
      if df.empty:
          df = pd.read_csv(f'rawdata/socgraph_{char}.csv')
      else:
          df = pd.concat([df, pd.read_csv(f'rawdata/socgraph_{char}.csv')])
    except:
        logs.append(char)
        pass

In [13]:
df = df.drop('Unnamed: 0', axis=1)

In [14]:
df.to_csv('socgraph_comb.csv')

In [15]:
top2spec = pd.DataFrame(df['topic'].unique(), columns=['topic'])
top2spec['specialty'] = None

In [16]:
top2spec.to_csv('top2spec.csv', index=False)

In [17]:
top2spec

,topic,specialty
0,Abdominal Disorders,None
1,Abscess - Dental,None
2,Abscess - Non-dental,None
3,Abuse,None
4,Accidents and Injuries,None
...,...,...
1288,Zolmitriptan,None
1289,Zoloft,None
1290,Zolpidem,None
1291,Zopiclone,None
